In [8]:
import pandas as pd
import numpy as np

In [9]:
def weightedKnnImpute(X, k):
    #Input: X:= pandas dataframe with uknown values set to NaN
    #       k:= number of nearest neighbors sought
    
    numberOfFeatures = X.shape[1]
    
    for index1, row1 in X.iterrows():
        #track progress
        print("row: " + str(index1))
        
        for i in range(numberOfFeatures):
            if np.isnan(row1[i]):

                #calculate distances
                distArray = []
                for index2, row2 in X.iterrows():
                    distance = 0
                    if(index1 != index2):
                        for j in range(numberOfFeatures):
                            if( not np.isnan(row1[j]) and not np.isnan(row2[j])):
                                distance += np.square(X.loc[index1, j] - row2[j])

                        #invert distance
                        distArray.append(1 / np.sqrt(distance))
                    else:
                        distArray.append(0)

                #find nearest neighbors and weight their values 
                result = 0
                values = []
                totalWeight = 0
                weights = []
                knn = k
                
                for l in range(knn):
                    argmax = np.argmax(distArray)
                    if not np.isnan(X.loc[argmax, i]):
                        values.append(X.loc[argmax, i])
                        totalWeight += distArray[argmax]
                        weights.append(distArray[argmax])
                        distArray[argmax] = 0
                    else:
                        distArray[argmax] = 0
                        knn += 1
   
                for m in range(len(values)):
                    result += values[m] * (weights[m] / totalWeight)

                row1[i] = result
                
    return X
    

In [ ]:
for i in range(1,4):
    # put data in pandas dataframe
    data = 'datasets/MissingValueEstimation/MissingData{}.txt'.format(i)
    X = pd.read_csv(data, sep='\s+', header=None)
    
    X = X.transpose()
    
    # header
    print('\n''*** MissingData{} ***'''.format(i))
    print('Number of Samples: ' + str(X.shape[0]))
    numberOfFeatures = X.shape[1]
    print('Number of Features: ' + str(X.shape[1]))
    print("\nOriginal Data: \n")

    # set uknowns in dataset to NaN
    X = X[X < 1.00000000000000e+99]
    print(str(X))
    
    Xresult = weightedKnnImpute(X, 4)
    
    Xresult = Xresult.transpose()
    
    np.savetxt("Results/LynchMissingResults{}.txt".format(i), Xresult.values, fmt='%f', delimiter="\t")
    